### Requirements

- langchain>=0.0.200
- openai>=0.27.8
- chromadb>=0.4.7
- beautifulsoup4>=4.12.2
- selenium>=4.11.2
- webdriver-manager>=3.8.6
- python-dotenv>=1.0.0
- pprintpp>=0.4.0

## 트러블 슈팅
---
**데이터 전처리 과정**
1. WebBaseLoader 로 레퍼런스가 정상적으로 인식되지 않는 문제  
 - 원인 : WebBaseLoader의 한계로 인한 실패 가능성. WebBaseLoader는 간단한 텍스트 추출에 적합하며, 복잡한 HTML 구조나 JavaScript 기반의 동적 콘텐츠를 처리하지 못할 수 있다. 

    1_1. WebBaseLoader 대신 request를 사용한 HTML 크롤링 시도 -> 실패  
     - 원인 : Notion 페이지는 JavaScript를 활용한 동적 렌더링을 사용하는 구조여서 requests만 사용해서는 JavaScript로 렌더링된 내용을 가져올 수 없었다.

    1_2. Selenium 사용 --> 부분 성공  
     - 원인 : implicitly_wait 으로 대기 후 로드 시 데이터를 가져오는 경우가 있고, 못가져오는 경우가 있었음..

    1_3. 대기 방식 변경 (WebDriverWait.until() 사용) --> 성공  
     - 로드하려는 웹페이지의 html 중 특정 요소가 로드될 때까지 wait하는 방식 사용

2. soup.get_text() 에서 노션 문서의 헤드라인만 추출되는 문제  
 - 원인 : 노션 페이지에 진입했을 때, 토글(Ctrl+Alt+T)이 닫힌 상태로 텍스트가 추출되고 있었음

    2_1. 단축키 입력을 선언하는 방법 (헤드리스 상태에서 사용 불가)  

    2_2. html 내에 토글 버튼을 찾아 여는 방법 (헤드리스 상태에서 사용 가능) --> 선택 (성공)  
     - drive.find_elements 로 버튼 탐색 후 클릭 (지연시간 1초)

3. 추출된 텍스트 파일에 일부 문법 구문이 남아있는 문제  
 - re (regular expression) 을 활용해 삭제/전처리 (성공)

---
**AI 챗봇**

1. AI 가 질문에 대한 사용자의 답변을 인식하지 못하는 문제  
 - 사용자의 답변을 질문과 함께 feedback_prompt 형식으로 묶어서 다시 AI에게 전달하는 방식으로 문제 해결

2. AI 가 동일한 주제에 대해서 반복적으로 비슷한 질문만을 하는 문제 --> (해결중..)



In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pprint import pprint

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

from dotenv import load_dotenv
import os

# .env 파일에서 환경변수 로드
load_dotenv("C:/.env")

llm = ChatOpenAI(model="gpt-4o-mini")

# Selenium 옵션 설정 (헤드리스 모드로 실행)
chrome_options = Options()
chrome_options.add_argument("--headless")  # 브라우저 창을 띄우지 않음
chrome_options.add_argument("--disable-gpu")  # GPU 비활성화 (일부 환경에서 필요)

# WebDriver 경로 설정 (자동 설치)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

url_list=[]
txt_list=[]

# 환경변수에 저장된 URL 로드
for i in range(1, 17):  # URL_1 ~ URL_16
    url = os.getenv(f"DL_URL_{i}")
    if url:  # 환경변수가 존재하면 추가
        url_list.append(url)

# 웹페이지 요청
for url in url_list:
    driver.get(url)  # 페이지 로드

    # 특정 요소가 로드될 때까지 기다림 (예: Notion 페이지에서 주요 콘텐츠가 담길 요소)
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".notion-page-content"))
        )
    except TimeoutException:
        print(f"페이지 로딩 실패: {url}")
        continue
    
    # 토글이 닫혀 있으면 토글을 열기
    try:
        # 모든 토글 버튼을 찾음 (Ctrl+Alt+T에 해당하는 토글을 찾아서 열기)
        toggle_buttons = driver.find_elements(By.XPATH, "//div[@role='button' and contains(@aria-label, '열기')]")
        
        # 각 토글을 클릭하여 열기
        for button in toggle_buttons:
            button.click()
            time.sleep(1)  # 토글이 열리기 전에 잠깐 대기
        
    except Exception as e:
        print(f"토글을 여는 데 실패했습니다: {e}")

    # 페이지의 HTML 가져오기
    html_code = driver.page_source

    # BeautifulSoup으로 HTML 파싱
    soup = BeautifulSoup(html_code, 'html.parser')

    txt = soup.get_text()

    # 1. \xa0를 공백으로 변환
    txt = txt.replace('\xa0', ' ')

    # 2. 정규식을 사용해 \\로 시작하는 LaTeX 명령어 제거
    txt = re.sub(r'\\[a-zA-Z]+\{.*?\}', '', txt)  # \command{...} 형식 제거
    txt = re.sub(r'\\[a-zA-Z]+', '', txt)        # \command 형식 제거

    # 3. 불필요한 공백 제거 (코드 개행 유지를 위해 주석처리)
    # txt = re.sub(r'\s+', ' ', txt).strip()

    # 텍스트만 가져오기
    txt_list.append(txt)


driver.quit()  # 브라우저 종료


In [15]:

# 결과 출력
pprint(txt_list[0])  # 두 번째 URL의 텍스트 내용 출력


('[스파르타코딩클럽] 1. 딥러닝 개념을 잡아봅시다![SCC] 기초가 탄탄한 딥러닝/[스파르타코딩클럽] 기초가 탄탄한 딥러닝 - '
 '1주차/[스파르타코딩클럽] 1. 딥러닝 개념을 잡아봅시다!제작:[스파르타코딩클럽] 1. 딥러닝 개념을 잡아봅시다![수업 목표]딥러닝이 '
 '무엇인지 개념에 대해 알아봅시다.딥러닝의 역사와 어디에 사용할 수 있을지 알아봅시다[목차]01. 딥러닝이란 무엇일까요?02. 딥러닝의 '
 '역사와 활용 방안03. 딥러닝을 배워야 하는 이유💡모든 토글을 열고 닫는 단축키\n'
 'Windows : Ctrl + alt + t \n'
 'Mac : ⌘ + ⌥ + t 01. 딥러닝이란 무엇일까요?✔️딥러닝이란 무엇인지 개념에 대해서 알아봅시다!1) 딥러닝이란? 딥러닝 '
 '개념딥러닝은 인공신경망(Artificial Neural Networks)을 기반으로 한 기계 학습의 한 분야입니다.다층 신경망을 사용하여 '
 '데이터로부터 특징을 자동으로 학습하고, 이를 통해 복잡한 문제를 해결합니다.입력 데이터에서 중요한 패턴을 추출하고, 이를 바탕으로 예측, '
 '분류, 생성 등의 다양한 작업을 수행할 수 있습니다.ALT 딥러닝의 특징비선형 추론: 딥러닝은 비선형 추론을 통해 복잡한 데이터의 패턴을 '
 '학습할 수 있습니다.다층 구조: 여러 층의 신경망을 사용하여 데이터의 고차원 특징을 학습합니다.자동 특징 추출: 데이터로부터 중요한 '
 '특징을 자동으로 추출하여 별도의 특징 공학(feature engineering) 과정이 필요 없습니다.02. 딥러닝의 역사와 활용 '
 '방안✔️딥러닝의 역사와 어디에 딥러닝을 쓸 수 있을지 배워봅시다!1) 딥러닝의 역사와 발전 발전 과정ALT 인공지능, 머신러닝, 딥러닝의 '
 '관계인공지능(AI) : 인공지능은 인간의 지능을 모방하여 문제를 해결하는 기술을 의미합니다. AI는 규칙 기반 시스템부터 자율 학습 '
 '시스템까지 다양한 접근 방식을 포함합니다.머신러닝(ML) : 머신러닝은 데이터를 이용해 모

In [19]:
from langchain.schema import Document

# 1. 로드된 문서 전처리(청킹)
docs = ''.join(txt_list)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
str_splits = text_splitter.split_text(docs)

# 2. 문자열 리스트를 Document 객체로 변환
doc_splits = [Document(page_content=str) for str in str_splits]

vectorstore = Chroma.from_documents(documents=doc_splits, embedding=OpenAIEmbeddings())
print(f"Number of document chunks: {len(doc_splits)}")

# 상위 10개의 청크 출력
print("Top 10 chunks:")
for i, chunk in enumerate(doc_splits[:10], 1):
    pprint(f"\nChunk {i}:\n{chunk.page_content}")

retriever = vectorstore.as_retriever()
prompt = ChatPromptTemplate.from_messages([("system", """
    당신은 AI 강사입니다. 아래 context를 기반으로 하나의 퀴즈를 만들어 사용자의 대답을 기다리세요.
    퀴즈는 보기가 있는 객관식 또는 O,X 형태로 출제해주세요. (주로 코드 내용과 관련된 문제를 추천합니다.)
    이후, 사용자의 대답을 확인하고 아래 형식을 바탕으로 피드백을 제공하세요:
    - 정답 여부: "N번" 또는 "예/아니오"
    - 추가 설명: (정답과 관련된 추가 정보를 제공하세요)
    
    Context: {context}
    """)])

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
        {"context": retriever | format_docs}
        | prompt
        | llm
        | StrOutputParser()
)

Number of document chunks: 365
Top 10 chunks:
('\n'
 'Chunk 1:\n'
 '[스파르타코딩클럽] 1. 딥러닝 개념을 잡아봅시다![SCC] 기초가 탄탄한 딥러닝/[스파르타코딩클럽] 기초가 탄탄한 딥러닝 - '
 '1주차/[스파르타코딩클럽] 1. 딥러닝 개념을 잡아봅시다!제작:[스파르타코딩클럽] 1. 딥러닝 개념을 잡아봅시다![수업 목표]딥러닝이 '
 '무엇인지 개념에 대해 알아봅시다.딥러닝의 역사와 어디에 사용할 수 있을지 알아봅시다[목차]01.')
('\n'
 'Chunk 2:\n'
 '수 있을지 알아봅시다[목차]01. 딥러닝이란 무엇일까요?02. 딥러닝의 역사와 활용 방안03. 딥러닝을 배워야 하는 이유💡모든 토글을 '
 '열고 닫는 단축키')
'\nChunk 3:\nWindows : Ctrl + alt + t'
('\n'
 'Chunk 4:\n'
 'Mac : ⌘ + ⌥ + t 01. 딥러닝이란 무엇일까요?✔️딥러닝이란 무엇인지 개념에 대해서 알아봅시다!1) 딥러닝이란? 딥러닝 '
 '개념딥러닝은 인공신경망(Artificial Neural Networks)을 기반으로 한 기계 학습의 한 분야입니다.다층 신경망을 사용하여 '
 '데이터로부터 특징을 자동으로 학습하고, 이를 통해 복잡한 문제를 해결합니다.입력')
('\n'
 'Chunk 5:\n'
 '통해 복잡한 문제를 해결합니다.입력 데이터에서 중요한 패턴을 추출하고, 이를 바탕으로 예측, 분류, 생성 등의 다양한 작업을 수행할 수 '
 '있습니다.ALT 딥러닝의 특징비선형 추론: 딥러닝은 비선형 추론을 통해 복잡한 데이터의 패턴을 학습할 수 있습니다.다층 구조: 여러 층의 '
 '신경망을 사용하여 데이터의 고차원 특징을 학습합니다.자동 특징 추출: 데이터로부터')
('\n'
 'Chunk 6:\n'
 '특징 추출: 데이터로부터 중요한 특징을 자동으로 추출하여 별도의 특징 공학(feature engineering) 과정이 필요 '
 '없습니다.02. 딥러닝의 역사

In [23]:

# 사용자가 "exit" 을 입력할 경우, 대화가 종료됩니다.
while True: 
    query = "퀴즈를 시작하세요"
    # if query.strip().lower() == "exit":
    #     print("대화를 종료합니다.")
    #     break
    
    # 1. 퀴즈 생성
    quiz = rag_chain.invoke(query)
    print("Generated Quiz:")
    print(quiz)
    
    # 2. 사용자 답변 수집
    user_answer = input("답변을 입력하세요: ")
    if user_answer.strip().lower() == "exit":
        print("대화를 종료합니다.")
        break
    print(user_answer)

    # 3. 사용자 답변에 대한 피드백 생성
    feedback_prompt = ChatPromptTemplate.from_messages([
        ("system", f"""
        AI 강사로서 다음 퀴즈의 정답 여부를 확인하고 피드백을 제공하세요.
        퀴즈: {quiz}
        사용자의 답변: {user_answer}
        피드백:
        """)
    ])
    feedback_chain = feedback_prompt | llm
    feedback = feedback_chain.invoke({"quiz": quiz, "answer": user_answer})
    print("Feedback:")
    pprint(feedback)



Generated Quiz:
퀴즈: 다음 중 모멘텀(Momentum) 기법에 대한 설명으로 옳은 것은 무엇일까요?

1. 모멘텀은 모델의 가중치를 무작위로 초기화하는 방법이다.
2. 모멘텀은 이전 기울기를 현재 기울기에 반영하여 학습 속도를 높인다.
3. 모멘텀은 학습 과정에서 진동을 증가시키기 위해 사용된다.
4. 모멘텀은 에포크 수를 결정하는 데 도움을 준다.

정답을 선택해 주세요 (1, 2, 3, 4 중 하나).
2번
Feedback:
AIMessage(content='정답: 2번\n\n피드백: 당신의 선택은 올바릅니다! 모멘텀(Momentum) 기법은 이전 기울기를 현재 기울기에 반영하여 학습 속도를 높이고, 경량화된 경로로 최적화할 수 있도록 돕습니다. 이는 특히 지역 최소값에 빠지는 문제를 완화하는 데 유용합니다. \n\n다른 선택지에 대한 간단한 설명도 추가하겠습니다:\n1. 모멘텀은 모델의 가중치를 무작위로 초기화하는 방법이 아닙니다. 가중치 초기화는 다른 기법을 통해 수행됩니다.\n3. 모멘텀은 오히려 학습 과정에서의 진동을 줄이는 데 도움을 줍니다.\n4. 모멘텀은 에포크 수를 결정하는 데 직접적인 도움을 주지 않습니다. 에포크 수는 주로 실험적 설정에 따라 결정됩니다.\n\n아주 잘 하셨습니다! 추가 질문이 있으면 언제든지 물어보세요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 224, 'prompt_tokens': 191, 'total_tokens': 415, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens':